In [61]:
from __future__ import print_function
from time import time
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD

from pymystem3 import Mystem

In [157]:
from nltk.corpus import stopwords

In [226]:
n_features = 5000
n_topics = 42
n_top_words = 3

In [227]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [228]:
stemmer = Mystem(entire_input=False)

In [229]:
dataset =  pd.read_csv('./NEWS/untitled/data.csv')
dataset.text.head()

0    Другой ключевой момент, на который работодател...
1    Министерство финансов России оценило «серый» ф...
2    Тема качества образования заслуживает отдельно...
3    09:00 — Пятигорск, отель «Хантама» Бросаем вещ...
4    В американских СМИ в последние месяцы не раз п...
Name: text, dtype: object

In [230]:
dataset.tokens = dataset.text.apply(stemmer.lemmatize)
dataset.clean_strings = dataset.tokens.apply(lambda x: ' '.join(t for t in x))

In [231]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.7, max_features=n_features, stop_words=stopwords.words('russian'), 
                                   ngram_range=[1,2])
tfidf_matrix = tfidf_vectorizer.fit_transform(dataset.clean_strings.values)

tf_vectorizer = CountVectorizer(max_df=0.7, max_features=n_features, stop_words=stopwords.words('russian'), 
                                ngram_range=[1,2])
tf_matrix = tf_vectorizer.fit_transform(dataset.clean_strings.values)

In [232]:
nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf_matrix)

In [233]:
print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)


Topics in NMF model:
Topic #0:
яндекс коммерческий комиссионный
Topic #1:
президент арб арб президент
Topic #2:
доллар рубль нефть
Topic #3:
банк финансовый рынок
Topic #4:
югра хотина хотин
Topic #5:
лимит малый малый средний
Topic #6:
карта чип терминал
Topic #7:
яндекс коммерческий комиссионный
Topic #8:
налог налоговый труд
Topic #9:
спфс swift сообщение
Topic #10:
яндекс коммерческий комиссионный
Topic #11:
заемщик ипотечный программа
Topic #12:
возмещение выплата вкладчик
Topic #13:
яндекс коммерческий комиссионный
Topic #14:
актив судебный процедура
Topic #15:
акция асв привилегированный
Topic #16:
руб млрд руб млрд
Topic #17:
яндекс коммерческий комиссионный
Topic #18:
трансформация платеж трансформация платеж
Topic #19:
яндекс коммерческий комиссионный
Topic #20:
госуслуга портал портал госуслуга
Topic #21:
блокчейн создание технология
Topic #22:
агаларов трамп сын
Topic #23:
яндекс коммерческий комиссионный
Topic #24:
мегафон евросеть вымпелком
Topic #25:
залог залоговый кре

In [234]:
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=15, learning_method='online',
                                learning_offset=50., random_state=42).fit(tf_matrix)


In [235]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0:
год финансовый swift
Topic #1:
интернет молодежь яндекс
Topic #2:
россия время наблюдать защита право
Topic #3:
банк сказать год
Topic #4:
снижаться московский биржа цср
Topic #5:
трамп агаларов сын
Topic #6:
рубль банк касперский
Topic #7:
lg lg cns cns
Topic #8:
крым турбина установка
Topic #9:
коллекция роман кольцо
Topic #10:
сегодняшний день который усиление
Topic #11:
банк инсайдерский информация пао
Topic #12:
pci dss pci dss
Topic #13:
акция асв привилегированный
Topic #14:
который год это
Topic #15:
год млн руб
Topic #16:
мвд мвд россия копейка
Topic #17:
банк год кредитный
Topic #18:
компания работать ваш
Topic #19:
банк который год
Topic #20:
год кредит акционерный общество
Topic #21:
год финансовый акция
Topic #22:
россия глава банк
Topic #23:
основной налоговый вычет
Topic #24:
диего tod сын
Topic #25:
пенсионный управляющий компания накопление
Topic #26:
спортсмен допинг уголовный
Topic #27:
спфс swift сообщение
Topic #28:
style style softl

In [240]:
from gensim.models import Word2Vec
from gensim.models import word2vec

In [256]:
dataset.clean_strings.head()

0    другой ключевой момент на который работодатель...
1    министерство финансы россия оценивать серый фо...
2    тема качество образование заслуживать отдельны...
3    пятигорск отель хантам бросать вещь у подножие...
4    в американский сми в последний месяц не раз по...
Name: text, dtype: object

In [318]:
w2v = Word2Vec(dataset.tokens.values, size=100, window=5, min_count=1)

In [319]:
w2v.most_similar(dataset.tokens.values[2])

[('или', 0.9999673366546631),
 ('получать', 0.9999666213989258),
 ('клиент', 0.999965488910675),
 ('продукт', 0.9999638199806213),
 ('страна', 0.999962568283081),
 ('другой', 0.999962329864502),
 ('проект', 0.9999619722366333),
 ('их', 0.9999613761901855),
 ('под', 0.9999609589576721),
 ('большой', 0.9999599456787109)]

In [261]:
w2v.most_similar(positive=['блокчейн'])

[('технология', 0.9998924136161804),
 ('механизм', 0.9998824596405029),
 ('продукт', 0.9998809695243835),
 ('позволять', 0.9998793601989746),
 ('клиент', 0.9998775720596313),
 ('для', 0.9998736381530762),
 ('использование', 0.9998728632926941),
 ('центр', 0.9998728036880493),
 ('развитие', 0.9998705983161926),
 ('проект', 0.99986732006073)]

In [345]:
dataset['clean_tokens'] = dataset.tokens.apply(lambda x: [t for t in x if t not in stopwords.words('russian')])

In [344]:
w2v_clean = Word2Vec(dataset.clean_tokens.values, size=100, window=5, min_count=1, sg=1, iter=100)

In [346]:
w2v_clean.most_similar(positive=['блокчейн'])

[('контрастировать', 0.6686022877693176),
 ('коллективно', 0.6667443513870239),
 ('применимость', 0.6626334190368652),
 ('специализироваться', 0.6576817035675049),
 ('наблюдательный', 0.6464658975601196),
 ('токен', 0.6444015502929688),
 ('работоспособность', 0.6435334086418152),
 ('оптимистичный', 0.640955924987793),
 ('записывать', 0.6324071884155273),
 ('функционирование', 0.6295121908187866)]

In [365]:
dataset.clean_tokens.values[11]

['однако',
 'несколько',
 'месяц',
 'назад',
 'сразу',
 'банк',
 'выводить',
 'рынок',
 'новый',
 'продукт',
 'карта',
 'рассрочка',
 'предполагать',
 'беспроцентный',
 'использование',
 'кредитный',
 'средство',
 'банка',
 'покупка',
 'магазин',
 'партнер',
 'нейминг',
 'оба',
 'карта',
 'закладывать',
 'глобальный',
 'месседж',
 'клиент',
 'это',
 'бесплатный',
 'деньги',
 'естественно',
 'возникать',
 'вопрос',
 'это',
 'нужный',
 'банк',
 'прибыль',
 'который',
 'формироваться',
 'данный',
 'случай',
 'исключительно',
 'счет',
 'комиссия',
 'партнерский',
 'организация',
 'платежный',
 'система',
 'безусловно',
 'размер',
 'прибыль',
 'несопоставимый',
 'процентный',
 'доход',
 'классический',
 'кредитный',
 'карта',
 'читать',
 'также',
 'халва',
 'совесть',
 'достоинство',
 'недостаток',
 'карта',
 'беспроцентный',
 'рассрочка',
 'продажа',
 'высокорискованный',
 'продукт',
 'минимальный',
 'комиссионный',
 'маржа',
 'сторона',
 'банк',
 'акт',
 'благотворительность',
 'продуманн

In [366]:
w2v_clean.most_similar(dataset.clean_tokens.values[11])

[('приветственный', 0.7696899771690369),
 ('порадовать', 0.7664345502853394),
 ('информировать', 0.7550635933876038),
 ('недалекий', 0.7321943640708923),
 ('лайн', 0.7181087136268616),
 ('PLUS', 0.7164790630340576),
 ('пополнение', 0.7055864334106445),
 ('автоматический', 0.7050371170043945),
 ('неслучайность', 0.7021297216415405),
 ('начисляться', 0.6988977193832397)]

In [350]:
w2v_clean.most_similar(negative=['блокчейн'])

[('благодаря', 0.07897493243217468),
 ('покупатель', 0.07555755972862244),
 ('аукцион', 0.06355968862771988),
 ('х', 0.06228058412671089),
 ('сносить', 0.059618547558784485),
 ('свыше', 0.05917669087648392),
 ('постройка', 0.05841321125626564),
 ('изымать', 0.05770673602819443),
 ('кстати', 0.05363631248474121),
 ('связывать', 0.053028032183647156)]

In [266]:
w2v.most_similar('доход')

[('сумма', 0.9999147653579712),
 ('от', 0.9999144673347473),
 ('покупка', 0.9999120831489563),
 ('около', 0.9999115467071533),
 ('на', 0.9999050498008728),
 ('объем', 0.9999037981033325),
 ('более', 0.9999030828475952),
 ('сша', 0.9999020099639893),
 ('уровень', 0.9999005794525146),
 ('программа', 0.9998966455459595)]

In [269]:
w2v.most_similar(positive=['миллиард', 'рубль'])

[('млрд', 0.9998084306716919),
 ('составлять', 0.9997642636299133),
 ('до', 0.9997141361236572),
 ('млн', 0.9997061491012573),
 ('год', 0.99969083070755),
 ('доллар', 0.9996715784072876),
 ('руб', 0.9996607899665833),
 ('сумма', 0.9995851516723633),
 ('на', 0.9995599389076233),
 ('около', 0.9995480179786682)]

In [270]:
w2v_clean.most_similar(positive=['миллиард', 'рубль'])

[('составлять', 0.9998921155929565),
 ('млрд', 0.9998754858970642),
 ('доллар', 0.99986732006073),
 ('год', 0.9998632073402405),
 ('млн', 0.9998360872268677),
 ('сумма', 0.9998263716697693),
 ('доход', 0.9998089075088501),
 ('около', 0.9998023509979248),
 ('курс', 0.9997999668121338),
 ('объем', 0.9997997283935547)]

In [371]:
w2v_clean.most_similar('AI')

[('отличать', 0.7938493490219116),
 ('уметь', 0.7344980239868164),
 ('сегментирование', 0.6891836524009705),
 ('продвинутый', 0.6886470317840576),
 ('продвигаться', 0.6854407787322998),
 ('журавль', 0.6804898977279663),
 ('робот', 0.6724709868431091),
 ('инжиниринг', 0.6625609397888184),
 ('персонализировать', 0.6602671146392822),
 ('зрительский', 0.6546989679336548)]

In [375]:
wiki = Word2Vec.load_word2vec_format('./NEWS/ruwikiruscorpora_0_300_20.bin', binary=True)

In [381]:
from copy import deepcopy

In [382]:
new_vocab = {}
for k in wiki.vocab.keys():
    new_vocab[k.split('_')[0]] = deepcopy(wiki.vocab[k])

In [383]:
wiki.vocab = new_vocab

In [393]:
wiki.index2word = [i.split('_')[0] for i in wiki.index2word]

In [395]:
wiki.most_similar('блокчейн')

[('ethereum', 0.6526474356651306),
 ('bitcoin', 0.6368981003761292),
 ('криптовалюта', 0.5974207520484924),
 ('litecoin', 0.5675651431083679),
 ('биткойн', 0.5644757747650146),
 ('ssl', 0.5278358459472656),
 ('транзакционный', 0.5243838429450989),
 ('direct::connect', 0.5220121741294861),
 ('oauth', 0.5208117365837097),
 ('биткоин', 0.5185723304748535)]

In [ ]:
wiki.most_similar()